In [32]:
using LinearAlgebra,SparseArrays
using DataFrames
using JuMP, Ipopt, Mosek,MosekTools,MathOptInterface

In [33]:
BranchData = DataFrame(From = [ 1, 1, 2, 3, 3, 4], To = [ 2, 3, 4, 4, 5, 5], 
    R = [ 0.0, 0.023, 0.006, 0.020, 0.0, 0.0], X = [ 0.3, 0.145, 0.032, 0.260, 0.320, 0.500], 
    Gsh = [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Bsh = [ 0.0, 0.040, 0.010, 0.0, 0.0, 0.0], 
    T = [ 1.0, 1.0, 1.0, 1.0, 0.98,1], Tl = [1.0, 1.0, 1.0, 1.0, 0.95, 1.0], Tu = [1.0, 1.0, 1.0, 1.0, 1.05, 1.0],
    ϕ = [ 0.0, 0.0, 0.0, -3.0, 0.0, 0.0], ϕl = [ 0.0, 0.0, 0.0, -30.0, 0.0, 0.0], ϕu = [ 0.0, 0.0, 0.0, 30.0, 0.0, 0.0]
    );
ZshB = [ 0.0+0.0*im 0.0+0.3*im 0.05+0.0*im 0.0+0.0*im 0.0+0.0*im]; 
GshB = [ 0.0 0.0 0.05 0.0 0.0];
BshB = [ 0.0 0.3 0.0 0.0 0.0];
BusNum = 5;
nLine= nrow(BranchData);

In [34]:
R = BranchData.R; X = BranchData.X; Bsh = BranchData.Bsh; T = BranchData.T; ϕ = BranchData.ϕ;
Y = conj((R + X*im).^-1); G = real(Y); B = imag(Y);
insertcols!(BranchData, 4, Y = round.(Y,digits = 3));
insertcols!(BranchData, 5, G = round.(G,digits = 3));
insertcols!(BranchData, 6, B = round.(B,digits = 3));
select!(BranchData, Not(:R));
select!(BranchData, Not(:X));
BranchData

,From,To,Y,G,B,Gsh,Bsh,T,Tl
,Int64,Int64,Complex…,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2,0.0+3.333im,0.0,3.333,0.0,0.0,1.0,1.0
2,1,3,1.067+6.727im,1.067,6.727,0.0,0.04,1.0,1.0
3,2,4,5.66+30.189im,5.66,30.189,0.0,0.01,1.0,1.0
4,3,4,0.294+3.824im,0.294,3.824,0.0,0.0,1.0,1.0
5,3,5,0.0+3.125im,0.0,3.125,0.0,0.0,0.98,0.95
6,4,5,0.0+2.0im,0.0,2.0,0.0,0.0,1.0,1.0


In [35]:
Y   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Y,BusNum,BusNum));
G   = real(Y); B = imag(Y);
Gsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Gsh,BusNum,BusNum));
Bsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Bsh,BusNum,BusNum));

In [36]:
T   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.T,BusNum,BusNum));
ϕ   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.ϕ,BusNum,BusNum));

In [37]:
BusData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PL = [ 0.0, 0.0, 0.0, 0.90, 0.239], QL = [ 0.0, 0.0, 0.0, 0.400, 0.129], 
    Vmin = [ 1.0, 0.95, 0.95, 0.95, 0.95], Vmax = [ 1.0, 1.05, 1.05, 1.05, 1.05]
    );
PL   = BusData.PL;
QL   = BusData.QL;
Vmin = BusData.Vmin;
Vmax = BusData.Vmax;

In [38]:
GenData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PGmin = [ 0.0, 0.0, 0.10, 0.05, 0.0], PGmax = [ 1.0, 0.0, 0.40, 0.40, 0.0], 
    QGmin = [ -0.4, 0.0, -0.2, -0.2, 0.0], QGmax = [ 0.4, 0.0, 0.30, 0.20, 0.0]
    );
PgL = GenData.PGmin;
PgU = GenData.PGmax;
QgL = GenData.QGmin;
QgU = GenData.QGmax;

In [39]:
model = nothing
model = Model(with_optimizer(Mosek.Optimizer,QUIET=false,MSK_IPAR_INFEAS_REPORT_AUTO=true))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

In [40]:
@variable(model, Pg[1:BusNum]);
@variable(model, Qg[1:BusNum]);
@variable(model, P[1:BusNum,1:BusNum]);
@variable(model, Q[1:BusNum,1:BusNum]);

In [41]:
@variable(model, W[i = 1:2*BusNum,j = 1:2*BusNum]);

In [42]:
for i in 1:BusNum
    set_upper_bound(Pg[i], PgU[i]);
    set_lower_bound(Pg[i], PgL[i]);
    set_upper_bound(Qg[i], QgU[i]);
    set_lower_bound(Qg[i], QgL[i]);
end

for i in findall(x->x==0, GenData.PGmax)
    delete_upper_bound(Pg[i])
    delete_lower_bound(Pg[i])
    delete_upper_bound(Qg[i])
    delete_lower_bound(Qg[i])
    fix(Pg[i],0)    
    fix(Qg[i],0)

end

In [43]:
@objective(model, Min, 0.35*Pg[1] + 0.2*Pg[3] + 0.4*Pg[3]*Pg[3] + 0.3*Pg[4] + 0.5*Pg[4]*Pg[4])

0.4 Pg[3]² + 0.5 Pg[4]² + 0.35 Pg[1] + 0.2 Pg[3] + 0.3 Pg[4]

In [44]:
@constraint(model,SOC_Con1[i = 1:BusNum ],   0  <= W[i,i] );
@constraint(model,SOC_Con2[j = 1:BusNum  ],  0  <= W[j,j] );
@constraint(model,SOC_Con3[i = 1:BusNum,j = 1:BusNum], [W[i,i],W[j,j], 1*W[i,j], 1*W[i+BusNum,j] ] in RotatedSecondOrderCone());

@constraint(model,SD_Con1[i = 1:BusNum,j = 1:BusNum ],   W[i,j+BusNum]  == -W[i+BusNum,j] );
@constraint(model,SD_Con2[i = 1:BusNum,j = 1:BusNum ],   W[i,j]  == W[i+BusNum,j+BusNum] );

In [45]:
@constraint(model, V_Con[i = 1:BusNum], Vmin[i]*Vmin[i]<=W[i,i]<=Vmax[i]*Vmax[i] ); 

In [46]:
δL = -80*π/180; δU = 80*π/180;
@constraint(model, δL_Con[i = 1:BusNum, j = 1:BusNum], tan(δL)*(W[i,j])<= W[i+BusNum,j] );  
@constraint(model, δU_Con[i = 1:BusNum, j = 1:BusNum], W[i+BusNum,j] <= tan(δU)*(W[i,j]) ); 

In [47]:
#@constraint(model,Sym_Con[i = 1:BusNum,j = 1:BusNum ],   W[i,j]  == W[j,i] );

In [48]:
@constraint(model, ActivePFij[i = 1:BusNum,j = 1:BusNum], P[i,j] ==
    ( G[i,j]*W[i,i] ) - ( G[i,j]*W[i,j] ) + ( B[i,j]*W[i+BusNum,j] ) );

In [49]:
#@expression(model, ActivePFij[i = 1:BusNum,j = 1:BusNum], 
#    ( G[i,j]*W[i,i] ) - ( G[i,j]*W[i,j] ) + ( B[i,j]*W[i+BusNum,j] ) );

In [50]:
@constraint(model, ActiveBranch[i = 1:BusNum],Pg[i]-PL[i] == 
sum(P[i,j] for j in 1:BusNum) );

In [51]:
#@constraint(model, ActiveBranch[i = 1:BusNum],Pg[i]-PL[i] == 
#sum(ActivePFij[i,j] for j in 1:BusNum) );

In [52]:
@constraint(model, ReActivePFij[i = 1:BusNum,j = 1:BusNum], Q[i,j] ==
    ( B[i,j]*W[i,i] ) - ( G[i,j]*W[i+BusNum,j] ) - ( B[i,j]*W[i,j] ) );

In [53]:
#@expression(model, ReActivePFij[i = 1:BusNum,j = 1:BusNum], 
#    ( B[i,j]*W[i,i] ) - ( G[i,j]*W[i+BusNum,j] ) - ( B[i,j]*W[i,j] ) );

In [54]:
@constraint(model, ReActiveBranch[i = 1:BusNum],Qg[i]-QL[i] == 
sum(Q[i,j] for j in 1:BusNum) );

In [55]:
#@constraint(model, ReActiveBranch[i = 1:BusNum],Qg[i]-QL[i] == 
#sum(ReActivePFij[i,j] for j in 1:BusNum) );

In [56]:
@constraint(model, ActiveLogic[i = 1:BusNum,j = 1:BusNum],P[i,j] == -P[j,i]);
@constraint(model, ReActiveLogic[i = 1:BusNum,j = 1:BusNum],Q[i,j] == -Q[j,i]);

In [57]:
optimize!(model);
termination_status(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 329             
  Cones                  : 26              
  Scalar variables       : 265             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 55
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 5
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective s

OPTIMAL::TerminationStatusCode = 1

In [67]:
objective_value(model)

0.3833375648164463

### Check Rank of W

In [68]:
x = value.(W)
rank(x,rtol=10^-5)

10

### Check Positive SemiDefinite(ness)

In [73]:
size(x)[1] == length(filter(p -> p >=0  ,real(eigen(x).values)))

false

In [76]:
value.(P)

5×5 Array{Float64,2}:
 -1.66533e-16   0.451548   0.449591     -0.0          -0.0        
 -0.451548     -0.0       -0.0           0.451548     -0.0        
 -0.449591      0.0       -1.11022e-16   0.381593      0.2555     
  0.0          -0.451548  -0.381593      1.00614e-16  -0.0164998  
  0.0           0.0       -0.2555        0.0164998     1.04083e-17